In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

save_path = "."

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])
print(df)

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values #regression

#build the neural network
model = Sequential()
model.add(Dense(25,input_dim=x.shape[1],activation='relu'))#hidden layer 1
model.add(Dense(10,activation = 'relu'))#hidden layer 2
model.add(Dense(1)) #output

model.compile(loss='mean_squared_error',optimizer = 'adam')
model.fit(x,y,verbose=2,epochs=100)

#predict
pred = model.predict(x)

#measure RMSE error\
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"Before save score(RMSE) : {score}")

#save neural network to JSON
model_json = model.to_json()
with open(os.path.join(save_path,"network.json"),"w") as json_file:
    json_file.write(model_json)
    
#save neurla network to yaml
model_yaml = model.to_yaml()
with open(os.path.join(save_path,"network.yaml"),"w") as yaml_file:
    yaml_file.write(model_yaml)

      mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0    18.0          8         307.0       130.0    3504          12.0    70   
1    15.0          8         350.0       165.0    3693          11.5    70   
2    18.0          8         318.0       150.0    3436          11.0    70   
3    16.0          8         304.0       150.0    3433          12.0    70   
4    17.0          8         302.0       140.0    3449          10.5    70   
..    ...        ...           ...         ...     ...           ...   ...   
393  27.0          4         140.0        86.0    2790          15.6    82   
394  44.0          4          97.0        52.0    2130          24.6    82   
395  32.0          4         135.0        84.0    2295          11.6    82   
396  28.0          4         120.0        79.0    2625          18.6    82   
397  31.0          4         119.0        82.0    2720          19.4    82   

     origin                       name  
0         1  chevrolet

## the above neural network is saved and if we want to reload it we use the below code

In [27]:
from tensorflow.keras.models import load_model
model2 = load_model(os.path.join(save_path,"network.h5"))
pred = model2.predict(x)
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"After Load score(RMSE) : {score}")

After Load score(RMSE) : 3.6991830969790147
